# Recipe 14: Build a Data Timeline

This notebook demonstrates how to analyze when data was collected for a target over time.

## What You'll Learn
- Extract and parse temporal metadata
- Group data by time periods
- Create timeline visualizations
- Find data gaps and coverage periods

In [ ]:
import pds.peppi as pep
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

## Query Mars Data

In [ ]:
client = pep.PDSRegistryClient()

# Get Mars data
products = pep.Products(client) \
    .has_target("Mars") \
    .observationals()

# Convert to DataFrame
df = products.as_dataframe(max_rows=500)

print(f"Retrieved {len(df)} products")

## Parse Temporal Data

In [ ]:
if df is not None and 'pds:Time_Coordinates.pds:start_date_time' in df.columns:
    # Convert to datetime
    df['start_date'] = pd.to_datetime(
        df['pds:Time_Coordinates.pds:start_date_time'],
        errors='coerce'
    )
    
    # Remove invalid dates
    df = df.dropna(subset=['start_date'])
    
    print(f"Valid dates found: {len(df)}")
    print(f"Earliest: {df['start_date'].min()}")
    print(f"Latest: {df['start_date'].max()}")

## Group by Year

In [ ]:
df['year'] = df['start_date'].dt.year
timeline = df.groupby('year').size()

print("Mars data products by year:")
print(timeline)

## Visualize Timeline

In [ ]:
plt.figure(figsize=(12, 6))
timeline.plot(kind='bar')
plt.xlabel('Year')
plt.ylabel('Number of Products')
plt.title('Mars Data Products Timeline (Sample)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Monthly Resolution Timeline

In [ ]:
df['year_month'] = df['start_date'].dt.to_period('M')
monthly_timeline = df.groupby('year_month').size()

plt.figure(figsize=(14, 6))
monthly_timeline.plot(kind='line')
plt.xlabel('Month')
plt.ylabel('Number of Products')
plt.title('Mars Data Products by Month (Sample)')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Key Takeaways

- Temporal metadata enables timeline analysis
- pandas datetime handling makes time-based grouping easy
- Visualization reveals data collection patterns and gaps
- Can analyze at different resolutions (year, month, day)

## Next Steps

- Compare timelines across different instruments
- Identify mission phases by data density
- Correlate with mission events
- Analyze data collection cadence